In [1]:
import os
import numpy as np
import sklearn
# from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.utils import Sequence
# from keras.utils import to_categorical
import sys


sys.path.append('..')

from preprocess.spectrogram import plot_mel_spect
# from preprocess.wav_helper import trim_audio_to_np_float

from preprocess.preprocess import preprocess, make_spects

from model.load import load_model


2025-10-25 20:28:02.825024: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-25 20:28:02.960216: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-25 20:28:06.724911: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
LABEL_NAMES = ['3S', 'BC', 'BD', 'BE', 'BhBl', 'BlBh', 'XlB', 'XsB']

In [3]:
model = load_model('../../.tstdata/models/zdenda-resnet.keras')

I0000 00:00:1761417103.334597  215526 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1583 MB memory:  -> device: 0, name: NVIDIA GeForce MX550, pci bus id: 0000:2b:00.0, compute capability: 7.5


In [4]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 192000)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mel_spectrogram                 │ (None, 128, 376)       │             0 │
│ (MelSpectrogram)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mel_to_magma (MelToMagma)       │ (None, 128, 376, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet152v2 (Functional)        │ (None, 2048)           │    58,331,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │        16,392 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 174,756,634 (666.64 MB)

 Trainable params: 58,204,296 (222.03 MB)

 Non-trainable params: 143,744 (561.50 KB)

 Optimizer params: 116,408,594 (444.06 MB)

In [ ]:
FP = "../../.tstdata/F002413.wav"

samples = preprocess(FP)

for sample in samples:
    # X = np.zeros((1, *spect.shape, 1), dtype=np.float32)
    # X[0, :, :, 0] = spect

    audio_tensor = tf.convert_to_tensor(
        np.asarray(sample).reshape(1, -1), dtype=tf.float32
    )

    prediction = model.predict(
        audio_tensor
    )


    pred_percent = dict(zip(LABEL_NAMES, map(lambda x: f"{round(float(x), 2) * 100}%", prediction.flatten())))
    percent_str = ' | '.join([ f"{k}:{pred_percent[k]}" for k in pred_percent.keys() ])

    print(percent_str)
    # plot_mel_spect(, title=percent_str)